In [1]:
import sys
# sys.stdout = open('logfile', 'w')
sys.path.append('..')

import attack
import data_loader as dl
import geopandas as gp
import pandas as pd
from tqdm import tqdm
import numpy as np
import libpysal
import itertools
from joblib import Parallel, delayed
import random
import matplotlib.pyplot as plt
from skmob.tessellation import tilers
import matplotlib.pyplot as plt
import seaborn as sns
from statannotations.Annotator import Annotator
%matplotlib inline
plt.rcParams.update({'figure.figsize':(10,5), 'figure.dpi':200})
sns.set_palette(sns.color_palette("colorblind"))

In [2]:
# Load preprocessed data 
raw_full_trip_gdf, raw_trip_sp_gdf, raw_trip_ep_gdf, tesselation_gdf = dl.load_geolife(data_type='raw', 
                                                                                       min_n_trips_per_user=1,
                                                                                       rand_n_week_period=None,
                                                                                       tessellation_diameter=200,
                                                                                       min_trip_length=200,
                                                                                       min_nr_points=50,
                                                                                       upper_quantile_length=0.95,
                                                                                       only_2008=True)
assert len(raw_full_trip_gdf) == len(raw_trip_sp_gdf) == len(raw_trip_ep_gdf)

# Merge Start Points (SP) and End Points (EP) with Tessellation
gdf_sp, gdf_ep = attack.match_boundary_points_with_tessellation(raw_trip_sp_gdf, raw_trip_ep_gdf, tesselation_gdf)

# Extract Full Trips that Start and End within Tessellation Area
full_trip_gdf, trip_sp_gdf, trip_ep_gdf, gdf_sp, gdf_ep = attack.extract_trips_that_start_end_in_tessellation(raw_full_trip_gdf, raw_trip_sp_gdf, raw_trip_ep_gdf, gdf_sp, gdf_ep)


Reading raw geolife geojson file...
Done.
Number of trajectories in loaded dataset: 5400
Number of users in loaded dataset: 73
Number of trips that start and end wihin tessellation area: 5101
Number of trips outside and therefore dropped: 299


In [28]:
full_trip_gdf.groupby('PERSON_ID').nunique().TRIP_ID.describe()

count     72.000000
mean      70.847222
std       87.658213
min        1.000000
25%       11.250000
50%       46.500000
75%       87.250000
max      494.000000
Name: TRIP_ID, dtype: float64

In [25]:
full_trip_gdf.TRIP_LEN_IN_MTRS.describe()

count     5101.000000
mean     13354.763176
std      14528.771405
min        202.738029
25%       2839.112467
50%       8338.111997
75%      18273.843041
max      81934.432104
Name: TRIP_LEN_IN_MTRS, dtype: float64

In [27]:
full_trip_gdf.TRIP_DURATION_IN_MINS.describe()

count    5101.000000
mean      140.264739
std       216.632775
min         1.366667
25%        17.166667
50%        42.116667
75%       158.666667
max      2085.283333
Name: TRIP_DURATION_IN_MINS, dtype: float64